In [0]:
import pandas as pd

In [0]:
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from sklearn import metrics
import sys
import warnings
warnings.filterwarnings('ignore')

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

import re


SEED = 42
np.random.seed(SEED)

In [0]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
stop_words=nltk.corpus.stopwords.words('english')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path = ('drive/My Drive/imdb_master.csv')

In [0]:
corpus = pd.read_csv(path, index_col=0, encoding='latin-1')
corpus = corpus.drop(["file"], axis=1)

In [0]:
test = corpus.loc[corpus["type"] == "test"].drop(["type"], axis=1)
train = corpus.loc[corpus["type"] == "train"].drop(["type"], axis=1)
unsup_texts = list(train.loc[train["label"] == "unsup", "review"])
train = train.loc[train["label"] != "unsup"]


In [0]:
train.to_csv("imdb_train.csv", encoding="utf-8")
test.to_csv("imdb_test.csv", encoding="utf-8")
corpus.to_csv("imdb_corpus.csv", encoding="utf-8")

In [0]:
train['label'] 

25000    neg
25001    neg
25002    neg
25003    neg
25004    neg
        ... 
49995    pos
49996    pos
49997    pos
49998    pos
49999    pos
Name: label, Length: 25000, dtype: object

In [0]:
len(test), len(train)

(25000, 25000)

In [0]:
train['label'] = train['label'].apply(lambda x : 0 if x=='neg' else 1)
test['label'] = test['label'].apply(lambda x : 0 if x=='neg' else 1)

In [0]:
train

,review,label
25000,Story of a man who has unnatural feelings for ...,0
25001,Airport '77 starts as a brand new luxury 747 p...,0
25002,This film lacked something I couldn't put my f...,0
25003,"Sorry everyone,,, I know this is supposed to b...",0
25004,When I was little my parents took me along to ...,0
...,...,...
49995,"Seeing as the vote average was pretty low, and...",1
49996,"The plot had some wretched, unbelievable twist...",1
49997,I am amazed at how this movie(and most others ...,1
49998,A Christmas Together actually came before my t...,1


In [0]:
lemmatizer = WordNetLemmatizer()

In [0]:
nltk.download('wordnet')
def tokenize(text):
    return token.tokenize(text)

def clear(text):
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text
    
def normalize(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(clear(text))]
    return words

train['processed'] = train.review.apply(lambda x: normalize(x))
test['processed'] = test.review.apply(lambda x: normolize(x))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
max_features = 10000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train['processed'])
list_tokenized_train = tokenizer.texts_to_sequences(train['processed'])

maxlen = 150
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = train['label']

embed_size = 256
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(40, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 200
epochs = 2

In [0]:
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/2
20000/20000 [==============================] - 170s 9ms/step - loss: 0.4897 - acc: 0.7604 - val_loss: 0.6384 - val_acc: 0.7336
Epoch 2/2
20000/20000 [==============================] - 166s 8ms/step - loss: 0.2176 - acc: 0.9169 - val_loss: 0.6989 - val_acc: 0.7158


In [0]:
list_tokenized_test = tokenizer.texts_to_sequences(test['processed'])
maxlen = 150
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [0]:
def prediction(x):
    if x>=0.4:
        return 1
    else:
        return 0
preds = [take_pred(x[0]) for x in model.predict(X_test)]

#**Accuracy** **score**

In [0]:
(predictions == test['label'].values).mean()

0.85672